##  1. Imports

In [1]:
import requests
import numpy as np
import json
import pandas as pd
import sys
import collections
import scipy
import matplotlib.pyplot as plt
from PIL import Image
import ibmseti
import os,sys
sys.path.append('/home/cs341seti/cs341-ibm-seti/')
import numpy as np
import pandas as pd
import ibmseti
import collections
import scipy.io
import matplotlib.pyplot as plt
import commonutils as cu
import PIL
from PIL import Image
import model_specs
import sklearn
from sklearn import svm

Using TensorFlow backend.


## 2. Gathered the File list store in a data frame

In [ ]:
# Downloading and saving all files to local
r = requests.get('https://dal.objectstorage.open.softlayer.com/v1/AUTH_cdbef52bdf7a449c96936e1071f0a46b/simsignals_files/public_list_headers_3april_2017.txt')
fileList = r.text.split('\n')[:-1]
fileListJSON = [json.loads(entry) for entry in fileList]
# Create a data frame with all the file info
fileListTuples = [(str(j['file_name']),str(j['signal_classification']),str(j['uuid'])) for j in fileListJSON]
fileListDF = pd.DataFrame.from_records(fileListTuples,columns=["file_name","signal_classification","uuid"])
# Creating a file index. Will use this for filenames for easier reference
fileListDF['file_index'] = fileListDF.index
fileListDF['file_index'] = fileListDF.file_index.apply(lambda x: str.zfill(str(x),6) )
## Assigning numbers to classes
labelMap = {'noise':0,'brightpixel':1,'narrowband':2,
            'narrowbanddrd':3,'squarepulsednarrowband':4,'squiggle':5,'squigglesquarepulsednarrowband':6}
fileListDF["label"] = fileListDF["signal_classification"].apply(lambda x: labelMap[x])
print labelMap
fileListDF.to_csv("fileList.csv",index=False)

## 3a. Functionality to download and save the Welch periodogram as a numpy array

In [ ]:
# Function: input - JSON line with url, output - spectrogram array
def downloadSaveSignalData(DFrow,base_url,container,data_path):
    #********************Configuration Parameters Start************************#
    # This is the expected pixel width of the output image, height is determined automatically
    desiredSignalWidth = 512
    assert desiredSignalWidth&31 == 0 , "Desired Image width should be a multiple of 32"
    # How much should should the periodogram frequency be scaled by determines periodogram length
    periodogramFrequencyScaling = 4
    # rotatePSD - accounts for the 90 degree rotation caused by the Welch Periodogram
    rotatePsd = True
    # Normalize PSD
    NormalizePsd = True
    # Gauss Filter PSD
    GaussFilterPsd = True
    # Number of columns eliminated fromthe periodogram to preven the vertical line around the center frequency
    numEliminatedColumns = 2
    #********************Configuration Parameters End**************************#
    # Access file via HTTP method
    fname = DFrow['file_name']
    print "\r{}".format(DFrow["file_index"]),
    r = requests.get('{}/{}/{}'.format(base_url, container, fname))   
    aca = ibmseti.compamp.SimCompamp(r.content)
    originalSignal = aca.get_signal()
    # 2. Get the signal dimensions
    rawSignalLength = originalSignal.shape[0]
    # 3. Finding the closest multiple of 32 for final image width
    desiredSignalWidth = ((desiredSignalWidth)&(~31))
    #  This is also the segment length of the periodoogram(num frequency bins)
    npersegIn = desiredSignalWidth
    # 4. Signal width as determined by the frequency scaling needed for noise rejection
    signalWidth = desiredSignalWidth * periodogramFrequencyScaling+numEliminatedColumns
    # Finding the scaled signal length to accommodate width as a multiple of desiredSignalWidth*periodogramFrequencyScaling and height as a multiple of 32
    scaledSignalLength = signalWidth * ((rawSignalLength/signalWidth)&(~31))
    #print(scaledSignalLength)
    signalHeight = scaledSignalLength/signalWidth
    scaledSignal = originalSignal[0:scaledSignalLength]
    # 5. Reshaping the scaled signal
    reshapedSignal = aca._reshape(scaledSignal,(signalHeight,signalWidth))
    # 6. Computing the periodogoram
    from scipy import signal
    welchPeriodogram = signal.welch(reshapedSignal,fs= 8000, nperseg=npersegIn,return_onesided=False)
    powerSpectralDensity = welchPeriodogram[1]
    # 7. Rotating the periodogoram + gauss filtering + normalizing
    rotatedPSD = np.hstack((powerSpectralDensity[:,256:(desiredSignalWidth - numEliminatedColumns/2)],powerSpectralDensity[:,numEliminatedColumns/2:desiredSignalWidth/2]))
    rotatedPSDGaussFiltered = rotatedPSD
    if(GaussFilterPsd):
        rotatedPSDGaussFiltered = scipy.ndimage.filters.gaussian_filter(rotatedPSD,sigma=1,order=0,mode="nearest")        
    rotatedNormalizedPSDGaussFiltered = rotatedPSDGaussFiltered
    if(NormalizePsd):
        rotatedNormalizedPSDGaussFiltered = rotatedPSDGaussFiltered/np.linalg.norm(rotatedPSDGaussFiltered)
    np.save("{}/{}.npy".format(data_path,DFrow['file_index']),rotatedNormalizedPSDGaussFiltered) 

## 3b. Download all periodogram data, save as a set of numpy arrays (skip if already exists)

In [ ]:
base_url = 'https://dal.objectstorage.open.softlayer.com/v1/AUTH_cdbef52bdf7a449c96936e1071f0a46b'
container = 'simsignals'
data_path = "../data/specdataraw"
### Iterate through fileListDF and save the periodogram of the signal data as .npy file
### Use file_index for filename
temp = fileListDF.apply(downloadSaveSignalData,axis=1,args=(base_url,container,data_path))
del temp

## 4. Converting all periodograms(numpy files) to images (skip if already exists)

In [ ]:
#********************Configuration Parameters Start************************#
numEliminatedColumns = 2
desiredImageWidth = 224
desiredImageHeight = 224
overwriteFiles = False
#********************Configuration Parameters End**************************#
specPath = "../data/specdataraw/"
specfiles = [f for f in os.listdir(specPath) if os.path.isfile(os.path.join(specPath, f))]
for i in range(len(specfiles)):
    fname = specfiles[i].split(".")[0]
    savePath = "../data/specdataimages_{}x{}/".format(desiredImageWidth,desiredImageHeight)
    if(not os.path.isfile('{}{}.jpeg'.format(savePath,fname))):
        # Convert and save each file as image
        print('Creating {}{}.jpeg\n'.format(savePath,fname))
        spec = np.load("{}{}.npy".format(specPath,fname))
        fig, ax = plt.subplots(figsize=(20, 10))
        ax.axis('off')
        ax.frameon = False
        fig.frameon = False
        ax.imshow((spec), aspect = 0.5*float(spec.shape[1]) / spec.shape[0],cmap="viridis",interpolation='nearest')
        # Saving the plot
        print("{}{}.png".format(savePath,fname))
        plt.savefig("{}{}.png".format(savePath,fname))
        # reopening 
        from PIL import Image
        img = Image.open("{}{}.png".format(savePath,fname))
        width = img.size[0]
        height = img.size[1]
        cropped_img = img.crop((200, 100, width-200, height-100))
        cropped_img = cropped_img.resize((desiredImageWidth,desiredImageWidth), Image.LANCZOS)
        cropped_img.save('{}{}.jpeg'.format(savePath,fname))
        plt.close('all')
    else:
        print('{}{}.jpeg Exists, skipping \n'.format(savePath,fname))
        

## 5. Generating activations from images using VGG net

In [ ]:
# Point to directory
# Call model activations function
readDir = "../data/specdataimages_224x224_rgb_periodogram/"
writeDir = "../data/specdataactivations_224x224_rgb_periodogram/"
layerName = "block5_pool"
poolfitSize = None
cu.datautils.generateAllActivations(readDir,writeDir,layerName,poolfitSize)

## 6. Creating the dataset from the activations

In [ ]:
## Read in all files from directory and combine them into train/val/test datasets
readDir = "../data/specdataactivations_224x224_rgb_periodogram/"
nvalidation = 3000
ntest = 1500
cu.datautils.createDataset(readDir,"fileList.csv",nvalidation,ntest,'../data/activations_224x224.h5',loadImages=False)

## 7. Loading the stored dataset with activations on a subset of classes - 4

In [2]:
## Read in all files from directory and combine them into train/val/test datasets
# Loading in a dataset with a subset of all classes
subsetClasses = {0.0:0.0,2.0:1.0,3.0:2.0,5.0:3.0}
dataset = cu.datautils.loadDataset("../data/activations_224x224.h5",subsetClasses=subsetClasses)

Dim of data: 25088
Number of training images = 7171
Number of validation images = 1903
Number of test images = 913
Distribution in training data:
0 - 1435
1 - 1453
2 - 2854
3 - 1429
Distribution in validation data:
0 - 388
1 - 362
2 - 753
3 - 400
Distribution in test data:
0 - 175
1 - 182
2 - 388
3 - 168


## Creating the model

In [11]:
def runLinSVMModel(dataset,C,nDataset,modeltype,printReports=True,gamma=None):
    x_train = dataset['x_train']
    y_train = dataset['y_train']
    x_test = dataset['x_test']
    y_test = dataset['y_test']
    
    # Scaling training and test data
    means = np.mean(x_train,axis=0)
    stddev = np.std(x_train,axis=0)
    # Preventing zero division
    stddev[stddev<1e-3] = 1
    x_train = (x_train - means)/stddev
    x_test = (x_test - means)/stddev
    
    if modeltype=='linSVM':
        lin_clf = svm.LinearSVC(C=C/nDataset,verbose=True,class_weight='balanced')
        lin_clf.fit(x_train, y_train)
        pred_train = lin_clf.predict(x_train)
        pred_test = lin_clf.predict(x_test)
    elif modeltype=='linSVR':
        lin_clf = svm.LinearSVC(C=C/nDataset,verbose=True)
        lin_clf.fit(x_train, y_train)
        pred_train = np.round(lin_clf.predict(x_train))
        pred_test = np.round(lin_clf.predict(x_test))
    elif modeltype=='rbfSVM':
        lin_clf = svm.SVC(C=C/nDataset,gamma=gamma,verbose=True,class_weight='balanced',
                          decision_function_shape='ovr')
        lin_clf.fit(x_train, y_train)
        pred_train = lin_clf.predict(x_train)
        pred_test = lin_clf.predict(x_test)

    train_report = sklearn.metrics.classification_report(y_train,pred_train)
    test_report = sklearn.metrics.classification_report(y_test,pred_test)

    train_confmat = sklearn.metrics.confusion_matrix(y_train,pred_train)
    test_confmat = sklearn.metrics.confusion_matrix(y_test,pred_test)
    
    if printReports:
        print train_report
        print train_confmat
        print test_report
        print test_confmat

        print("Classification accuracy: %0.2f" % sklearn.metrics.accuracy_score(y_test,pred_test) )
        print("MSE: %0.2f" % np.mean(np.square(y_test - lin_clf.predict(x_test))) )
        print("Predictions correlation: %0.2f") % np.corrcoef(y_test,pred_test,rowvar=0)[0,1]
    
    result = {'lin_clf':lin_clf,'train_report':train_report,'train_confmat':train_confmat,
             'test_report':test_report,'test_confmat':test_confmat,
             'train_score':lin_clf.score(x_train,y_train),
             'test_score':lin_clf.score(x_test,y_test),
             'y_test':y_test,
             'pred_test':pred_test}
    return result

## Running the model on this dataset

In [4]:
## Loading in and preparing datasets
x_train = dataset['x_train']
y_train = dataset['y_train']
x_val = dataset['x_val']
y_val = dataset['y_val']
x_test = dataset['x_test']
y_test = dataset['y_test']
num_val = dataset['x_val'].shape[0]
num_test = dataset['x_test'].shape[0]
nb_classes = 4
## Parameter search for SVM
C_values = 10**np.random.uniform(-3,0,10)
train_scores = []
test_scores = []
for C_val in C_values:
    a = runLinSVMModel(dataset,C_val,len(dataset['y_train']),printReports=False,modeltype='linSVM')
    train_scores.append(a['train_score'])
    test_scores.append(a['test_score'])
    print "Results for C={}:".format(C_val)
    print "Train score={}   Test score={}".format(a['train_score'],a['test_score'])

[LibLinear]Results for C=0.798483018903:
Train score=0.935294937944   Test score=0.676889375685
[LibLinear]Results for C=0.352225523093:
Train score=0.853576906986   Test score=0.683461117196
[LibLinear]Results for C=0.591126494036:
Train score=0.906428671036   Test score=0.672508214677
[LibLinear]Results for C=0.0040945648008:
Train score=0.671036117696   Test score=0.661555312158
[LibLinear]Results for C=0.0103990001293:
Train score=0.686096778692   Test score=0.684556407448
[LibLinear]Results for C=0.00233495338377:
Train score=0.661274578162   Test score=0.658269441402
[LibLinear]Results for C=0.144739674261:
Train score=0.774926788453   Test score=0.684556407448
[LibLinear]Results for C=0.474160998872:
Train score=0.884256031237   Test score=0.679079956188
[LibLinear]Results for C=0.620149044166:
Train score=0.911030539674   Test score=0.672508214677
[LibLinear]Results for C=0.866410257968:
Train score=0.941988565054   Test score=0.673603504929


## Analyzing the failures, using a C value with the best prediction from above

In [14]:
## Loading in and preparing datasets
x_train = dataset['x_train']
y_train = dataset['y_train']
x_val = dataset['x_val']
y_val = dataset['y_val']
x_test = dataset['x_test']
y_test = dataset['y_test']
num_val = dataset['x_val'].shape[0]
num_test = dataset['x_test'].shape[0]
nb_classes = 4
# Using the best C-val from the run above
C_val = 0.144739674261
train_scores = []
test_scores = []
a = runLinSVMModel(dataset,C_val,len(dataset['y_train']),printReports=False,modeltype='linSVM')
train_scores.append(a['train_score'])
test_scores.append(a['test_score'])
print "Results for C={}:".format(C_val)
print "Train score={}   Test score={}".format(a['train_score'],a['test_score'])
print a['y_test']
print a['pred_test']
print dataset['test_ids']
print (a['test_confmat'])

[LibLinear]Results for C=0.144739674261:
Train score=0.774926788453   Test score=0.684556407448
[ 2.  3.  0.  1.  2.  3.  2.  0.  1.  2.  2.  2.  3.  2.  1.  3.  0.  1.
  2.  0.  2.  2.  2.  1.  0.  1.  2.  2.  0.  0.  1.  3.  0.  2.  2.  0.
  2.  1.  1.  2.  3.  3.  1.  0.  2.  2.  0.  1.  2.  2.  1.  3.  1.  3.
  2.  2.  2.  2.  0.  3.  2.  2.  0.  1.  2.  0.  3.  1.  1.  2.  3.  2.
  2.  1.  2.  0.  2.  2.  2.  3.  2.  1.  1.  0.  2.  2.  1.  1.  2.  0.
  2.  1.  2.  3.  2.  2.  0.  0.  2.  3.  0.  1.  2.  2.  0.  2.  3.  2.
  3.  2.  2.  1.  0.  1.  2.  2.  2.  0.  2.  3.  1.  2.  2.  3.  3.  0.
  1.  2.  0.  2.  1.  3.  3.  0.  0.  1.  3.  2.  2.  1.  3.  1.  2.  2.
  2.  2.  3.  3.  1.  1.  2.  0.  3.  1.  1.  1.  1.  2.  0.  3.  2.  3.
  0.  3.  2.  0.  2.  1.  0.  0.  1.  1.  3.  3.  2.  1.  2.  2.  1.  1.
  2.  0.  2.  3.  2.  2.  0.  2.  3.  1.  0.  1.  0.  3.  0.  0.  2.  2.
  3.  3.  2.  2.  0.  0.  2.  3.  1.  1.  3.  1.  3.  0.  3.  2.  0.  3.
  2.  1.  3.  2.  2.  1.  0.